## Download AGN training set from ALeRCE-ZTF data base
  


In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import astropy.units as u
from astropy import coordinates
import astropy.io.fits as pf
from alerce_xmatch import Catalog, TargetCatalog, OutputCols, crossmatch

  
### Get the credentials:

In [2]:
import json
credentials_file = "alercereaduser.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

    print(params)

{'dbname': 'ztf', 'user': 'alerceread', 'host': 'psql.alerce.online', 'password': 'alerce2019'}


  
### Connect to DB

In [3]:
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])
cur = conn.cursor()

### Function to query data more easily

In [4]:
def sql_query(query):
    cur.execute(query)
    result = cur.fetchall()
    
    # Extract the column names
    col_names = []
    for elt in cur.description:
        col_names.append(elt[0])

    #Convert to dataframe
    df = pd.DataFrame(np.array(result), columns = col_names)
    return(df)

  
### Load AGN MILLIQUAS catalog

In [5]:
#mycatdf = pd.read_csv("../catalogs/Oh2015_Type1AGNsfromSDSSDR7.csv")

# rename some columns to be compatible with xmatch service
#mycatdf.rename({"RAJ2000": "ra", "DEJ2000": "dec"}, axis="columns", inplace=True)
#mycatdf.index.ID=["mycat_id"]

# drop nans in ra, dec
#mycatdf.dropna(subset=['ra', 'dec'], inplace=True)

#mycat = Catalog(df=mycatdf, name="type1AGN")

#mycatdf.head()

mycatdf = pd.read_csv("../catalogs/Other_inZTF_trainingset.csv")
mycatdf.head()

,oid,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
0,ZTF19abfivgh,Periodic-Other,314.324447,-25.715291,34.422223,CRTSsouth,SSS_J205717.9-254254,Rotational Var,1.959896
1,ZTF19abfhulf,Periodic-Other,334.220738,-24.898974,0.364151,CRTSsouth,SSS_J221652.9-245356,Rotational Var,0.697852
2,ZTF19abfbcvl,Periodic-Other,242.707508,-25.799313,0.277041,CRTSsouth,SSS_J161049.7-254757,Rotational Var,1.594802
3,ZTF19abfbcfr,Periodic-Other,243.718365,-25.231259,0.827283,CRTSsouth,SSS_J161452.4-251352,Rotational Var,0.210267
4,ZTF19abfbbtl,Periodic-Other,245.380133,-21.432053,0.455537,CRTSsouth,SSS_J162131.2-212555,Rotational Var,0.504236


   
### Do crossmatch between the two catalogs (MILLIQUAS and ZTF)   

In [6]:
#xmatch = mycat.crossmatch(TargetCatalog.ZTF, output=OutputCols.BOTH).df
# save the xmatch
#xmatch.to_csv("../catalogs/type1AGN_inZTF_trainingset_milliquas_Jun2019.csv", index=False)

  
### Save the light curves 

In [7]:
#oids = mycatdf['ZTF_oid'].values
#oids = xmatch['ZTF_oid'].values

oids = mycatdf['oid'].values

#conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])
#cur = conn.cursor()


nn=0
for oid in oids:
    nn+=1
    print(nn)
    query = "select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='"+str(oid)+"'"
    print(query)
    lc_data = sql_query(query)
    lc_data = lc_data.sort_values(['mjd'])
        
    lc_g = lc_data[lc_data['fid']==1]
    lc_r = lc_data[lc_data['fid']==2]

    #save the light curves in csv files, separated by band
    
    if len(lc_g['mjd'].values)>5 or len(lc_r['mjd'].values)>5:
        plt.figure(figsize=(10,10))

    
    if len(lc_g['mjd'].values)>5:
        
        lc_g.to_csv("../Other/%s_other_gband.csv" % (str(oid)), index=False)
        print(oid, " g band saved")

        plt.errorbar(lc_g['mjd'].astype(float) ,lc_g['magpsf_corr'].astype(float),yerr=lc_g['sigmapsf_corr'].astype(float), fmt='.-', color='blue',ecolor='blue',label='g')


    if len(lc_r['mjd'].values)>5:
        
        lc_r.to_csv("../Other/%s_other_rband.csv" % (str(oid)), index=False)
        print(oid, " r band saved")
        
        plt.errorbar(lc_r['mjd'].astype(float) ,lc_r['magpsf_corr'].astype(float),yerr=lc_r['sigmapsf_corr'].astype(float), fmt='.-', color='red',ecolor='red',label='r')


    
    if len(lc_g['mjd'].values)>5 or len(lc_r['mjd'].values)>5:
        plt.legend()
        plt.xlabel('MJD')
        plt.ylabel('magnitude')
        plt.gca().invert_yaxis()
        plt.savefig("../Other_plots/%s_other.png" % (str(oid)))
        plt.close('all')
    

1
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfivgh'
2
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfhulf'
3
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfbcvl'
4
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfbcfr'
5
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfbbtl'
6
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abfbbth'
7
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigma

56
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abagjcm'
57
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abagizv'
58
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abagixc'
59
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abadbyh'
60
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19abacosd'
61
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aazwyez'
62
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr

109
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayhqof'
ZTF19aayhqof  g band saved
110
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayhqhh'
ZTF19aayhqhh  g band saved
111
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayftjo'
112
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayftbo'
113
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayfssn'
114
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aayfsqq'
115
select mjd, 

161
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlibg'
162
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlias'
163
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlhyi'
164
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlhsr'
165
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlhny'
166
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aaxlhje'
167
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magp

214
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aauekdc'
215
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aatyrvw'
216
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aatyrsf'
ZTF19aatyrsf  g band saved
ZTF19aatyrsf  r band saved
217
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aatyrpw'
218
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aatyrlu'
219
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF19aatyrlq'
220
select mjd, 

...

469
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18acaccaa'
470
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abzzrvt'
ZTF18abzzrvt  r band saved
471
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abznqqg'
472
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abznpwn'
473
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abxpvil'
ZTF18abxpvil  g band saved
474
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abxbivf'
ZTF18abxbivf  r 

518
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abtlwoo'
ZTF18abtlwoo  g band saved
ZTF18abtlwoo  r band saved
519
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abthdze'
520
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abtdzlj'
ZTF18abtdzlj  r band saved
521
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18absygff'
522
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18absuujf'
523
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF

ZTF18abccpyw  g band saved
565
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abbwhdc'
566
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abbrogp'
ZTF18abbrogp  r band saved
567
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abafbtf'
568
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18abadjvf'
ZTF18abadjvf  g band saved
ZTF18abadjvf  r band saved
569
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aazztwr'
570
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 fro

ZTF18aabxdwd  g band saved
ZTF18aabxdwd  r band saved
612
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aabsiul'
ZTF18aabsiul  r band saved
613
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aabpzic'
ZTF18aabpzic  g band saved
ZTF18aabpzic  r band saved
614
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aabptwf'
ZTF18aabptwf  g band saved
ZTF18aabptwf  r band saved
615
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aabpsdb'
ZTF18aabpsdb  r band saved
616
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF18aabprue'
617
selec

ZTF17aacgpcx  g band saved
ZTF17aacgpcx  r band saved
656
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF17aacdeds'
657
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF17aabwosf'
ZTF17aabwosf  g band saved
ZTF17aabwosf  r band saved
658
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF17aabwftz'
659
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF17aabwfsa'
660
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_corr, distpsnr1, sgscore1 from detections where oid='ZTF17aabwfpq'
ZTF17aabwfpq  g band saved
661
select mjd, fid, ra, dec, magnr, isdiffpos, magpsf, sigmapsf, magpsf_corr, sigmapsf_co